# PROJETO - Big Data para Dados Públicos
## Professor Maciel Calebe
## Análise de Propagação de Doenças  
#### Arthur Boschini da Fonseca, Caio de Camargo Aranha Tieri, Guilherme Farias Caproni


Requistos minimos para rodar o código:  
* DASK
* DASK-ml
* SciKit-Learn
* Pandas
* Numpy


```pip install pandas```

```pip install numpy```

```pip install scikit-learn```

```pip install "dask[complete]"```

```pip install dask-ml```

## Introdução

O objetivo deste projeto é realizar uma análise de dados provenientes do governo que descrevem registros de atendimentos médicos com suspeita de gripe, de modo a treinar um modelo capaz de predigir a partir de sintomas se o caso tem potencial de se tornar grave ou não.

## Problema

A crescente incidência de casos de doenças respiratórias, especialmente em períodos sazonais, demanda uma abordagem proativa para a identificação precoce de casos graves. A problemática reside na complexidade dos dados governamentais disponíveis, que podem apresentar variabilidade e nuances importantes. Nesse contexto, a necessidade de desenvolver um modelo preditivo eficiente torna-se crucial, visando não apenas a prevenção e gestão adequada de casos de gripe, mas também a otimização dos recursos de saúde pública. A intricada relação entre sintomas e a gravidade do quadro destaca a importância de uma abordagem analítica robusta, capaz de extrair padrões significativos e fornecer insights valiosos para aprimorar a tomada de decisões na área da saúde.


## Leitura dos dados

##### Database Utilizada
*DATASUS - SRAG 2019*: Disponível em: https://opendatasus.saude.gov.br/dataset/srag-2019

##### Dicionário de Dados
Disponível em: https://d26692udehoye.cloudfront.net/SRAG/2019/dicionario_de_dados_SRAG_hospitalizado_2019.pdf




#### Importando as bibliotecas



In [80]:

# Bibliotecas DASK
import dask
from dask.distributed import Client
import dask.dataframe as dd # similar ao pandas
import dask.multiprocessing



import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from dask_ml.model_selection import train_test_split as dask_train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score


ModuleNotFoundError: No module named 'sklearn'

#### Criando o client do DASK

In [146]:
client = Client(n_workers=2, threads_per_worker=1, memory_limit='1GB')
client

c:\Users\FONSECA CERTO\anaconda3\lib\site-packages\distributed\node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60870 instead
  warnings.warn(


c:\Users\FONSECA CERTO\anaconda3\lib\site-packages\distributed\client.py:1274: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| numpy   | 1.23.2 | 1.23.2    | 1.21.6  |
| tornado | 6.2    | 6.2       | 6.1     |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:60870/status,
Dashboard: http://127.0.0.1:60870/status,Workers: 2
Total threads: 2,Total memory: 1.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:60871,Workers: 2
Dashboard: http://127.0.0.1:60870/status,Total threads: 2
Started: Just now,Total memory: 1.86 GiB
Comm: tcp://127.0.0.1:60896,Total threads: 1
Dashboard: http://127.0.0.1:60897/status,Memory: 0.93 GiB
Nanny: tcp://127.0.0.1:60876,


#### Lendo a base de dados

In [147]:
df = dd.read_csv('INFLUD19.csv', encoding='latin-1', sep=';', dtype= {'DT_NOTIFIC': 'string', 'SEM_NOT': 'int64', 'DT_SIN_PRI': 'O', 'SEM_PRI': 'int64', 'SG_UF_NOT': 'O', 'ID_REGIONA': 'O', 'CO_REGIONA': 'float64', 'ID_MUNICIP': 'O', 'CO_MUN_NOT': 'int64', 'ID_UNIDADE': 'O', 'CO_UNI_NOT': 'int64', 'CS_SEXO': 'O', 'DT_NASC': 'O', 'NU_IDADE_N': 'int64', 'TP_IDADE': 'int64', 'COD_IDADE': 'int64', 'CS_GESTANT': 'int64', 'CS_RACA': 'float64', 'CS_ESCOL_N': 'float64', 'ID_PAIS': 'O', 'CO_PAIS': 'float64', 'SG_UF': 'O', 'CO_RG_RESI': 'float64', 'ID_MN_RESI': 'O', 'CO_MUN_RES': 'float64', 'CS_ZONA': 'float64', 'SURTO_SG': 'float64', 'NOSOCOMIAL': 'float64', 'AVE_SUINO': 'float64', 'FEBRE': 'float64', 'TOSSE': 'float64', 'GARGANTA': 'float64', 'DISPNEIA': 'float64', 'DESC_RESP': 'float64', 'SATURACAO': 'float64', 'DIARREIA': 'float64', 'VOMITO': 'float64', 'OUTRO_SIN': 'float64', 'OUTRO_DES': 'O', 'PUERPERA': 'float64', 'CARDIOPATI': 'float64', 'HEMATOLOGI': 'float64', 'SIND_DOWN': 'float64', 'HEPATICA': 'float64', 'ASMA': 'float64', 'DIABETES': 'float64', 'NEUROLOGIC': 'float64', 'PNEUMOPATI': 'float64', 'IMUNODEPRE': 'float64', 'RENAL': 'float64', 'OBESIDADE': 'float64', 'OBES_IMC': 'float64', 'OUT_MORBI': 'float64', 'MORB_DESC': 'O', 'VACINA': 'float64', 'DT_UT_DOSE': 'O', 'MAE_VAC': 'float64', 'DT_VAC_MAE': 'O', 'M_AMAMENTA': 'float64', 'DT_DOSEUNI': 'O', 'DT_1_DOSE': 'O', 'DT_2_DOSE': 'O', 'ANTIVIRAL': 'float64', 'TP_ANTIVIR': 'float64', 'OUT_ANTIV': 'O', 'DT_ANTIVIR': 'O', 'HOSPITAL': 'float64', 'DT_INTERNA': 'O', 'SG_UF_INTE': 'O', 'ID_RG_INTE': 'O', 'CO_RG_INTE': 'float64', 'ID_MN_INTE': 'O', 'CO_MU_INTE': 'float64', 'UTI': 'float64', 'DT_ENTUTI': 'O', 'DT_SAIDUTI': 'O', 'SUPORT_VEN': 'float64', 'RAIOX_RES': 'float64', 'RAIOX_OUT': 'O', 'DT_RAIOX': 'O', 'AMOSTRA': 'float64', 'DT_COLETA': 'O', 'TP_AMOSTRA': 'float64', 'OUT_AMOST': 'O', 'REQUI_GAL': 'O', 'IF_RESUL': 'float64', 'DT_IF': 'O', 'POS_IF_FLU': 'float64', 'TP_FLU_IF': 'float64', 'POS_IF_OUT': 'float64', 'IF_VSR': 'float64', 'IF_PARA1': 'float64', 'IF_PARA2': 'float64', 'IF_PARA3': 'float64', 'IF_ADENO': 'float64', 'IF_OUTRO': 'float64', 'DS_IF_OUT': 'O', 'LAB_IF': 'O', 'CO_LAB_IF': 'float64', 'PCR_RESUL': 'float64', 'DT_PCR': 'O', 'POS_PCRFLU': 'float64', 'TP_FLU_PCR': 'float64', 'PCR_FLUASU': 'float64', 'FLUASU_OUT': 'O', 'PCR_FLUBLI': 'float64', 'FLUBLI_OUT': 'O', 'POS_PCROUT': 'float64', 'PCR_VSR': 'float64', 'PCR_PARA1': 'float64', 'PCR_PARA2': 'float64', 'PCR_PARA3': 'float64', 'PCR_PARA4': 'float64', 'PCR_ADENO': 'float64', 'PCR_METAP': 'float64', 'PCR_BOCA': 'float64', 'PCR_RINO': 'float64', 'PCR_OUTRO': 'float64', 'DS_PCR_OUT': 'O', 'LAB_PCR': 'O', 'CO_LAB_PCR': 'float64', 'CLASSI_FIN': 'float64', 'CLASSI_OUT': 'O', 'CRITERIO': 'float64', 'EVOLUCAO': 'float64', 'DT_EVOLUCA': 'O', 'DT_ENCERRA': 'O', 'DT_DIGITA': 'O', 'HISTO_VGM': 'int64', 'PAIS_VGM': 'float64', 'CO_PS_VGM': 'float64', 'LO_PS_VGM': 'O', 'DT_VGM': 'float64', 'DT_RT_VGM': 'float64', 'PCR_SARS2': 'float64', 'PAC_COCBO': 'float64', 'PAC_DSCBO': 'float64'})

In [148]:
df.head()

,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,...,DT_DIGITA,HISTO_VGM,PAIS_VGM,CO_PS_VGM,LO_PS_VGM,DT_VGM,DT_RT_VGM,PCR_SARS2,PAC_COCBO,PAC_DSCBO
0,10/01/2019,2,06/01/2019,2,MG,BELO HORIZONTE,1449.0,BELO HORIZONTE,310620,UNIDADE DE PRONTO ATENDIMENTO LESTE,...,14/03/2019,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,03/01/2019,1,01/01/2019,1,SP,GVE I CAPITAL,1331.0,SAO PAULO,355030,HOSP MUN M BOI MIRIM,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,02/01/2019,1,31/12/2018,1,PE,1,1497.0,RECIFE,261160,IMIP,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10/01/2019,2,07/01/2019,2,SP,GVE XVII CAMPINAS,1342.0,CAMPINAS,350950,HOSPITAL MUNICIPAL DR MARIO GATTI CAMPINAS,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11/01/2019,2,06/01/2019,2,PE,4,1499.0,CARUARU,260410,SECRETARIA MUNICIPAL DE SAUDE DE CARUARU,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Filtrando os dados 


In [149]:
informacoes_pessoais = df[["DT_NOTIFIC","CS_SEXO", "DT_NASC", "CS_RACA", "SG_UF", "SURTO_SG", "NOSOCOMIAL", ]]
informacoes_pessoais = informacoes_pessoais.rename(columns={"DT_NOTIFIC": "data_registro","CS_SEXO": "sexo", "DT_NASC": "data_nascimento", "CS_RACA": "raca", "SG_UF": "estado", "SURTO_SG": "surto_sindromegripal", "NOSOCOMIAL": "contraiu_no_hospital"})
informacoes_pessoais.head()

,data_registro,sexo,data_nascimento,raca,estado,surto_sindromegripal,contraiu_no_hospital
0,10/01/2019,M,17/03/1988,1.0,MG,2.0,2.0
1,03/01/2019,F,30/05/2018,1.0,SP,9.0,NaN
2,02/01/2019,M,07/05/2017,4.0,PE,9.0,9.0
3,10/01/2019,F,15/07/2018,1.0,SP,2.0,2.0
4,11/01/2019,F,15/09/2018,4.0,PE,9.0,9.0


Trocando as legendas do dataframe para valores normais

In [150]:
informacoes_pessoais['raca'] = informacoes_pessoais['raca'].replace({1.0: 'Branca', 2.0: 'Preta', 3.0: 'Amarela', 4.0: 'Parda', 5.0: 'Indígena', 9.0: 'Ignorado'})
informacoes_pessoais['surto_sindromegripal'] = informacoes_pessoais['surto_sindromegripal'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
informacoes_pessoais['contraiu_no_hospital'] = informacoes_pessoais['contraiu_no_hospital'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado', np.nan: 'Ignorado'})

Fazendo o mesmo para as informações de contágio

In [152]:
sintomas = df[["FEBRE", "TOSSE", "GARGANTA", "DISPNEIA", "DESC_RESP", "SATURACAO", "DIARREIA", "VOMITO",  "OUTRO_SIN", "OUTRO_DES"]]
sintomasbinario = df[["FEBRE", "TOSSE", "GARGANTA", "DISPNEIA", "DESC_RESP", "SATURACAO", "DIARREIA", "VOMITO",  "OUTRO_SIN", "OUTRO_DES"]]

In [153]:
sintomas['FEBRE'] = sintomas['FEBRE'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['TOSSE'] = sintomas['TOSSE'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['GARGANTA'] = sintomas['GARGANTA'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['DISPNEIA'] = sintomas['DISPNEIA'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['DESC_RESP'] = sintomas['DESC_RESP'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['SATURACAO'] = sintomas['SATURACAO'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['DIARREIA'] = sintomas['DIARREIA'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['VOMITO'] = sintomas['VOMITO'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['OUTRO_SIN'] = sintomas['OUTRO_SIN'].replace({1.0: 'Sim', 2.0: 'Não', 9.0: 'Ignorado'})
sintomas['OUTRO_DES'] = sintomas['OUTRO_DES'].replace({np.nan: 'sem descrição'})


In [154]:
sintomasbinario['FEBRE'] = sintomasbinario['FEBRE'].replace({1.0: 1, 2.0: 0, 9.0: 0})
sintomasbinario['TOSSE'] = sintomasbinario['TOSSE'].replace({1.0: 1, 2.0: 0, 9.0: 0})
sintomasbinario['GARGANTA'] = sintomasbinario['GARGANTA'].replace({1.0: 1, 2.0: 0, 9.0: 0})
sintomasbinario['DISPNEIA'] = sintomasbinario['DISPNEIA'].replace({1.0: 1, 2.0: 0, 9.0: 0})
sintomasbinario['DESC_RESP'] = sintomasbinario['DESC_RESP'].replace({1.0: 1, 2.0: 0, 9.0: 0})
sintomasbinario['SATURACAO'] = sintomasbinario['SATURACAO'].replace({1.0: 1, 2.0: 0, 9.0: 0})
sintomasbinario['DIARREIA'] = sintomasbinario['DIARREIA'].replace({1.0: 1, 2.0: 0, 9.0: 0})
sintomasbinario['VOMITO'] = sintomasbinario['VOMITO'].replace({1.0: 1, 2.0: 0, 9.0: 0})
sintomasbinario['OUTRO_SIN'] = sintomasbinario['OUTRO_SIN'].replace({1.0: 1, 2.0: 0, 9.0: 0})
sintomasbinario['OUTRO_DES'] = sintomasbinario['OUTRO_DES'].replace({np.nan: 0})

In [155]:
sintomas.head()

,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,OUTRO_DES
0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,DERRAME PLEURAL
1,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0
2,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0
3,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0
4,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0


In [156]:
sintomasbinario.head(10)

,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,OUTRO_DES
0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,DERRAME PLEURAL
1,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0
2,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0
3,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0
4,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0
5,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,ESPIRROS E CORIZA
6,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0
7,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0
8,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,CIANOSE
9,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0


Juntando as duas tabelas

In [157]:
#Juntando as duas tabelas: informacoes_pessoais e sintomas
df = dd.concat([informacoes_pessoais, sintomas], axis=1)
df.head()

c:\Users\FONSECA CERTO\anaconda3\lib\site-packages\dask\dataframe\multi.py:1260: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


,data_registro,sexo,data_nascimento,raca,estado,surto_sindromegripal,contraiu_no_hospital,mes_registro,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,OUTRO_DES
0,10/01/2019,M,17/03/1988,Branca,MG,Não,Não,1,Sim,Sim,Não,Sim,Sim,Sim,Não,Não,Sim,DERRAME PLEURAL
1,03/01/2019,F,30/05/2018,Branca,SP,Ignorado,Ignorado,1,Sim,Sim,Não,Sim,Sim,Sim,Não,Não,Não,sem descrição
2,02/01/2019,M,07/05/2017,Parda,PE,Ignorado,Ignorado,1,Sim,Sim,Não,Sim,Sim,Sim,Não,Não,Não,sem descrição
3,10/01/2019,F,15/07/2018,Branca,SP,Não,Não,1,Sim,Sim,Ignorado,Sim,Sim,Sim,Não,Não,Não,sem descrição
4,11/01/2019,F,15/09/2018,Parda,PE,Ignorado,Ignorado,1,Sim,Não,Não,Sim,Sim,Sim,Ignorado,Ignorado,Não,sem descrição


In [158]:
dfbinario = dd.concat([informacoes_pessoais, sintomasbinario], axis=1)

c:\Users\FONSECA CERTO\anaconda3\lib\site-packages\dask\dataframe\multi.py:1260: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


## Análise

Para criar o modelo de predição iremos utilizar técnica de predição Random Forest

Considerando como gripe a presença dos sintomas: febre, garganta, tosse e desconforto respiratório, ou a presença de
surto sindrome gripal

In [159]:
dfbinario['e_gripe'] = (df['FEBRE'] == 1) & (df['GARGANTA'] == 1) & (df['TOSSE'] == 1) & (df['DESC_RESP'] == 1) | (df['surto_sindromegripal'] == 'Sim')
dfbinario.head()

,data_registro,sexo,data_nascimento,raca,estado,surto_sindromegripal,contraiu_no_hospital,mes_registro,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,OUTRO_DES,e_gripe
0,10/01/2019,M,17/03/1988,Branca,MG,Não,Não,1,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,DERRAME PLEURAL,False
1,03/01/2019,F,30/05/2018,Branca,SP,Ignorado,Ignorado,1,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0,False
2,02/01/2019,M,07/05/2017,Parda,PE,Ignorado,Ignorado,1,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0,False
3,10/01/2019,F,15/07/2018,Branca,SP,Não,Não,1,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0,False
4,11/01/2019,F,15/09/2018,Parda,PE,Ignorado,Ignorado,1,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0,False


Verificando a quantidade de gripe no dataset, segundo os critérios mencionados

In [179]:
soma = 0
somafalse = 0
for i in dfbinario['e_gripe']:
    if pd.notna(i):
        if i == True:
            soma +=1
        else:
            somafalse +=1
print("quantidade de casos em que é gripe segundo os parâmetros:", soma)
print("quantidade total casos:", somafalse)

quantidade de casos em que é gripe segundo os parâmetros: 3102
quantidade total casos: 45426


#### Realizando o modelo de predição

Definindo que um caso grave de gripe é quando há todos os sintomas listados no dataset.

In [171]:
dfbinario['grave'] = ((dfbinario['FEBRE'] == 1) & (dfbinario['TOSSE'] == 1) & (dfbinario['GARGANTA'] == 1) & 
                      (dfbinario['DISPNEIA'] == 1) & (dfbinario['DESC_RESP'] == 1) & (dfbinario['SATURACAO'] == 1) & 
                      (dfbinario['DIARREIA'] == 1) & (dfbinario['VOMITO'] == 1)).astype(int)

Separando as colunas que serão utilizadas para a predição:

In [ ]:
sintomas_cols = ['FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'VOMITO']
dfbinario[sintomas_cols] = dfbinario[sintomas_cols].fillna(0).astype(int)

Como o dataset é grande, necessitamos usar o .compute() para que o dask compute o dataframe e possamos trabalhar com ele.

In [172]:
ddf_binario = dfbinario.compute()

Variáveis:

In [173]:
# Variáveis independentes
variavel_x = ddf_binario[sintomas_cols].values
# Variável alvo
variavel_y = ddf_binario['grave']

Treinando o modelo:

In [174]:
X_train, X_test, y_train, y_test = dask_train_test_split(variavel_x, variavel_y, test_size=0.2, random_state=42, shuffle=True)

modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

Colhendo os resultados:

In [175]:
y_pred = modelo.predict(X_test)

print("Acurácia:", accuracy_score(y_test, y_pred))
print("Relatório de classificação:")
print(classification_report(y_test, y_pred))

Acurácia: 1.0
Relatório de classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9659
           1       1.00      1.00      1.00        47

    accuracy                           1.00      9706
   macro avg       1.00      1.00      1.00      9706
weighted avg       1.00      1.00      1.00      9706



Verificando outras métricas:

In [176]:
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))
print("\nPrecisão:", precision_score(y_test, y_pred))
print("Revocação:", recall_score(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))

Matriz de Confusão:
[[9659    0]
 [   0   47]]

Precisão: 1.0
Revocação: 1.0
F1-Score: 1.0


#### Conclusão da predição do modelo

In [177]:
if precision_score(y_test, y_pred) == 1.0 and recall_score(y_test, y_pred) == 1.0:
    print("\nO modelo atingiu uma acurácia de 100%, indicando que todas as previsões foram corretas.\n Isso sugere uma capacidade robusta em identificar corretamente os casos graves de gripe.\n No entanto, é importante ressaltar que a acurácia perfeita pode ser um indicativo de sobreajuste,\n principalmente se o conjunto de dados não for representativo de casos do mundo real.")
else:
    print("\nO modelo atingiu uma acurácia alta, mas é necessário analisar outras métricas para avaliação mais detalhada. \nA precisão e a revocação indicam o desempenho do modelo em classificar corretamente os casos graves de gripe,\n mas pode haver espaço para melhorias, especialmente em cenários desbalanceados.\n É recomendável considerar ajustes no modelo, coletar mais dados ou explorar características adicionais para aprimorar a previsão.")


Conclusão: O modelo atingiu uma acurácia de 100%, indicando que todas as previsões foram corretas.
 Isso sugere uma capacidade robusta em identificar corretamente os casos graves de gripe.
 No entanto, é importante ressaltar que a acurácia perfeita pode ser um indicativo de sobreajuste,
 principalmente se o conjunto de dados não for representativo de casos do mundo real.


## Conclusão

Ao concluir o projeto, os resultados indicaram uma notável acurácia de 100% para o modelo desenvolvido, revelando sua eficácia em prever corretamente a gravidade dos casos de gripe analisados. Essa taxa de precisão sugere uma capacidade robusta do modelo em distinguir com precisão os casos graves, o que representa um avanço significativo na abordagem preditiva para atendimentos médicos relacionados à gripe. No entanto, é crucial considerar a ressalva de que uma acurácia perfeita pode, em alguns casos, ser um indicativo de sobreajuste. A perfeição nos resultados pode levantar preocupações sobre a generalização do modelo, especialmente se o conjunto de dados utilizado para treinamento não for representativo o suficiente dos casos reais observados na prática clínica. Portanto, a validação contínua do modelo em cenários do mundo real e a avaliação de sua eficácia em diferentes conjuntos de dados são passos essenciais para assegurar a confiabilidade e utilidade prática do modelo preditivo desenvolvido neste projeto.